In [1]:
import json
from py2neo import Graph, Node 

In [2]:
from pymongo import MongoClient

In [3]:
# connect to Mongo
client = MongoClient()
db = client.prova
movie_coll = db.movies
user_coll = db.users
movie_json = movie_coll.find({})
user_json = user_coll.find({})

In [4]:
# connect to graph
graph = Graph(uri="bolt://localhost:7687", auth=("neo4j", "qazqaz"))

In [5]:
# start a transaction
tx = graph.begin()
# all crud operations below can also be done on graph instead of tx

In [6]:
# open movie's json
with open('movies.json') as movies:
    movie_json = json.load(movies)

In [7]:
# renamings
for movie in movie_json:
    if "tid" in movie:
        movie["_id"] = movie["tid"]
        del movie["tid"]
    
    if "originaltitle" in movie:
        movie["original_title"] = movie["originaltitle"]
        del movie["originaltitle"]

In [8]:
# fix missing titles
for movie in movie_json:
    if movie["title"] is None:
        if movie["original_title"] is None:
            if movie["title_ita"] is None:
                print(movie["_id"])
            movie["title"] = movie["title_ita"]
        movie["title"] = movie["original_title"]

In [9]:
for x in movie_json:
    movie = {"id": x["_id"], "title": x["title"], "year": x["year"]}
    n = Node("Movie", **movie)
    #tx.create(n) 
    tx.merge(n, "Movie", "id")

In [10]:
# commit transaction
tx.commit()

In [11]:
# start a transaction
tx = graph.begin()
# all crud operations below can also be done on graph instead of tx

In [12]:
# open users's json
with open('users.json') as users:
    user_json = json.load(users)

In [13]:
# add a new node from dict
# Node(label, **dictionary)
for x in user_json:
    user = {"username": x["username"]}
    m = Node("User", **user)
    #m.__primarylabel__ = "User"
    #m.__primarykey__ = "username"
    #tx.create(m)
    tx.merge(m, "User", "username")

In [14]:
# commit transaction
tx.commit()